In [24]:
import tensorflow as tf
from sklearn.datasets import load_digits
import numpy as np

In [25]:
digits = load_digits()
#将数据改写成浮点数
X_data = digits.data.astype(np.float32)
Y_data = digits.target.astype(np.float32).reshape(-1,1)
print (X_data.shape)
print (Y_data.shape)

(1797, 64)
(1797, 1)


In [26]:
#导入标准化包
from sklearn.preprocessing import MinMaxScaler

In [27]:
scaler = MinMaxScaler()
#标准化数据
X_data = scaler.fit_transform(X_data)
from sklearn.preprocessing import OneHotEncoder
##one-hot编码
Y = OneHotEncoder().fit_transform(Y_data).todense() 
print(type(Y))

<class 'numpy.matrix'>


In [28]:
# 转换为图片的格式 （batch，height，width，channels）
X = X_data.reshape(-1,8,8,1)
print(X)
batch_size = 8 # 使用MBGD算法，设定batch_size为8

[[[[0.        ]
   [0.        ]
   [0.3125    ]
   ...
   [0.0625    ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.8125    ]
   ...
   [0.9375    ]
   [0.3125    ]
   [0.        ]]

  [[0.        ]
   [0.1875    ]
   [0.9375    ]
   ...
   [0.6875    ]
   [0.5       ]
   [0.        ]]

  ...

  [[0.        ]
   [0.25      ]
   [0.6875    ]
   ...
   [0.75      ]
   [0.4375    ]
   [0.        ]]

  [[0.        ]
   [0.125     ]
   [0.875     ]
   ...
   [0.75      ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.375     ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]]


 [[[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.3125    ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.5625    ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.1875    ]
   ...
   [0.375     ]
   [0.        ]
   [0.        ]]

  ...

  [[0.        ]
   [0.        ]
   [0.0

In [29]:
#定义一个生成批次函数
def generatebatch(X,Y,n_examples, batch_size):
    #定义一个循环处理
    for batch_i in range(n_examples // batch_size):
        start = batch_i*batch_size
        end = start + batch_size
        batch_xs = X[start:end]
        batch_ys = Y[start:end]
        yield batch_xs, batch_ys # 生成每一个batch

In [30]:
#从tensorflow框架中导入ops包
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
# 输入层
tf_X = tf.placeholder(tf.float32,[None,8,8,1])
tf_Y = tf.placeholder(tf.float32,[None,10])

In [31]:
# 卷积层+激活层
conv_filter_w1 = tf.Variable(tf.random_normal([3, 3, 1, 10]))
conv_filter_b1 =  tf.Variable(tf.random_normal([10]))
relu_feature_maps1 = tf.nn.relu(\
                tf.nn.conv2d(tf_X, conv_filter_w1,strides=[1, 1, 1, 1], padding='SAME') + conv_filter_b1)

In [32]:
# 池化层
max_pool1 = tf.nn.max_pool(relu_feature_maps1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

In [33]:
# 卷积层
conv_filter_w2 = tf.Variable(tf.random_normal([3, 3, 10, 5]))
conv_filter_b2 =  tf.Variable(tf.random_normal([5]))
conv_out2 = tf.nn.conv2d(relu_feature_maps1, conv_filter_w2,strides=[1, 2, 2, 1], padding='SAME') + conv_filter_b2
print (conv_out2)

Tensor("add_1:0", shape=(None, 4, 4, 5), dtype=float32)


In [34]:
# BN归一化层+激活层 
batch_mean, batch_var = tf.nn.moments(conv_out2, [0, 1, 2], keep_dims=True)
shift = tf.Variable(tf.zeros([5]))
scale = tf.Variable(tf.ones([5]))
epsilon = 1e-3
BN_out = tf.nn.batch_normalization(conv_out2, batch_mean, batch_var, shift, scale, epsilon)
print (BN_out)
relu_BN_maps2 = tf.nn.relu(BN_out)

Tensor("batchnorm/add_1:0", shape=(None, 4, 4, 5), dtype=float32)


In [35]:
# 池化层
max_pool2 = tf.nn.max_pool(relu_BN_maps2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

In [36]:
print (max_pool2)

Tensor("MaxPool_1:0", shape=(None, 2, 2, 5), dtype=float32)


In [37]:
# 将特征图进行展开
max_pool2_flat = tf.reshape(max_pool2, [-1, 2*2*5])

In [38]:
# 全连接层
fc_w1 = tf.Variable(tf.random_normal([2*2*5,50]))
fc_b1 =  tf.Variable(tf.random_normal([50]))
fc_out1 = tf.nn.relu(tf.matmul(max_pool2_flat, fc_w1) + fc_b1)

In [39]:
# 输出层
out_w1 = tf.Variable(tf.random_normal([50,10]))
out_b1 = tf.Variable(tf.random_normal([10]))
pred = tf.nn.softmax(tf.matmul(fc_out1,out_w1)+out_b1)

In [40]:
loss = -tf.reduce_mean(tf_Y*tf.log(tf.clip_by_value(pred,1e-11,1.0)))

In [41]:
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [42]:
y_pred = tf.arg_max(pred,1)
bool_pred = tf.equal(tf.arg_max(tf_Y,1),y_pred)

In [43]:
accuracy = tf.reduce_mean(tf.cast(bool_pred,tf.float32)) # 准确率

In [44]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1000): # 迭代1000个周期
        for batch_xs,batch_ys in generatebatch(X,Y,Y.shape[0],batch_size): # 每个周期进行MBGD算法
            sess.run(train_step,feed_dict={tf_X:batch_xs,tf_Y:batch_ys})
        if(epoch%100==0):
            res = sess.run(accuracy,feed_dict={tf_X:X,tf_Y:Y})
            print (epoch,res)
    res_ypred = y_pred.eval(feed_dict={tf_X:X,tf_Y:Y}).flatten() # 只能预测一批样本，不能预测一个样本
    print (res_ypred)

0 0.14802448
100 0.79243183
200 0.7957707
300 0.79632723
400 0.79632723
500 0.7957707
600 0.79632723
700 0.95937675
800 0.99109626
900 0.99499166
[0 1 2 ... 5 9 5]


In [45]:
from sklearn.metrics import  accuracy_score

In [46]:
print (accuracy_score(Y_data,res_ypred.reshape(-1,1)))

0.9938786867000556
